In [2]:
from qiskit.providers.fake_provider import FakeToronto

In [ ]:
from qiskit.providers.fake_provider import FakeToronto
# defexecute_on_real_machine(compiled_qobjs, shots=8192, machine_name='ibmq_paris', max_acceptable_circuits_by_device=900, repeats=1, execution_mode='Default'):
# Initialize the fake backend
device = FakeToronto()

## Replicate the qobjs based on shots 
post_compile_qobjs = []
for i in range(len(compiled_qobjs)):
    for _ in range(repeats): 
        post_compile_qobjs.append(compiled_qobjs[i])  # Simply replicate the qobjs as many times as required
# Determine the batch size 
_batch_size = min(max_acceptable_circuits_by_device, len(post_compile_qobjs))
batch_size = []
if (len(post_compile_qobjs) % _batch_size) != 0:
    num_batches = (len(post_compile_qobjs) // _batch_size) + 1
    for x in range(num_batches - 1):
        batch_size.append(_batch_size)
    batch_size.append(len(post_compile_qobjs) % _batch_size)
else:
    num_batches = len(post_compile_qobjs) // _batch_size
    for x in range(num_batches):
        batch_size.append(_batch_size)

print('Number of compiled qobjs:', len(compiled_qobjs))
print('Total number of circuits to be executed:', len(post_compile_qobjs))
print('Executed in batches of:', batch_size)

_machine_counts = [] 
job_ids = []
job_noise_properties = []
job_time = []
job_results = []

machine_shot_counts = shots
print('Running a total of', len(post_compile_qobjs), 'quantum objects in', num_batches, 'batches')
for batch_id in range(num_batches):
    quantum_objects = []
    for qc in range(batch_size[batch_id]):
        quantum_objects.append(post_compile_qobjs[int((batch_id * batch_size[batch_id]) + qc)])
    
    # Assemble the quantum objects for execution
    experiments = assemble(quantum_objects, backend=device, shots=machine_shot_counts)
    print('Status of Circuit Batch:', batch_id)
    
    # Run the job on the device and monitor it
    ibmq_job = device.run(experiments)
    job_monitor(ibmq_job)
    
    # Retrieve results from the job
    machine_results = ibmq_job.result()
    for qc in range(batch_size[batch_id]):
        _machine_counts.append(machine_results.get_counts(qc))
    
    # Log metadata
    job_results.append(machine_results)
    job_noise_properties.append(device.properties())
    job_ids.append(ibmq_job.job_id())
    job_time.append(ibmq_job.time_per_step())
# Re-organize the machine counts
machine_counts = []
for i in range(len(compiled_qobjs)):
    merged_dictionary = {}
    for j in range(repeats):
        required_dictionary_index = i * repeats + j  # Each qobj is repeated "repeats" number of times, so index of the dictionary is computed for entry into global list 
        merged_dictionary = update_dist(merged_dictionary, _machine_counts[required_dictionary_index]) 
    machine_counts.append(merged_dictionary)
    
# return machine_counts, job_ids, job_noise_properties, job_time, job_results

In [6]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.providers.fake_provider import FakeToronto
from qiskit import assemble
from qiskit.test.mock import FakeProvider
from qiskit.tools.monitor import job_monitor
device = FakeToronto()
def create_test_circuits(num_circuits=3, num_qubits=2):
    circuits = []
    for _ in range(num_circuits):
        qc = QuantumCircuit(num_qubits)
        qc.h(range(num_qubits))
        for i in range(num_qubits - 1):
            qc.cx(i, i + 1)
        qc.measure_all()
        circuits.append(qc)
    return circuits

def update_dist(existing_counts, new_counts):
    """ A simple function to merge quantum circuit execution counts. """
    for key, value in new_counts.items():
        if key in existing_counts:
            existing_counts[key] += value
        else:
            existing_counts[key] = value
    return existing_counts

# Define test parameters
shots = 8192
max_acceptable_circuits_by_device = 900
repeats = 1

# Create test circuits
compiled_qobjs = create_test_circuits(num_circuits=5, num_qubits=3)
device = FakeToronto()

# Simulate executing quantum objects
post_compile_qobjs = []
for i in range(len(compiled_qobjs)):
    for _ in range(repeats):
        post_compile_qobjs.append(compiled_qobjs[i])

_batch_size = min(max_acceptable_circuits_by_device, len(post_compile_qobjs))
batch_size = []
if (len(post_compile_qobjs) % _batch_size) != 0:
    num_batches = (len(post_compile_qobjs) // _batch_size) + 1
    for x in range(num_batches - 1):
        batch_size.append(_batch_size)
    batch_size.append(len(post_compile_qobjs) % _batch_size)
else:
    num_batches = len(post_compile_qobjs) // _batch_size
    for x in range(num_batches):
        batch_size.append(_batch_size)

# Execute the circuits in batches
_machine_counts = [] 
job_ids = []
job_noise_properties = []
job_time = []
job_results = []
machine_shot_counts = shots
print('Running a total of', len(post_compile_qobjs), 'quantum objects in', num_batches, 'batches')
for batch_id in range(num_batches):
    quantum_objects = []
    for qc in range(batch_size[batch_id]):
        quantum_objects.append(post_compile_qobjs[int((batch_id * batch_size[batch_id]) + qc)])
    
    experiments = assemble(quantum_objects, backend=device, shots=machine_shot_counts)
    ibmq_job = device.run(experiments)
    print('Status of Circuit Batch:', ibmq_job)
    # job_monitor(ibmq_job)
#     machine_results = ibmq_job.result()
#     for qc in range(batch_size[batch_id]):
#         _machine_counts.append(machine_results.get_counts(qc))
#     job_results.append(machine_results)
#     job_noise_properties.append(device.properties())
#     job_ids.append(ibmq_job.job_id())
#     job_time.append(ibmq_job.time_per_step())

# # Re-organize the machine counts
# machine_counts = []
# for i in range(len(compiled_qobjs)):
#     merged_dictionary = {}
#     for j in range(repeats):
#         required_dictionary_index = i * repeats + j
#         merged_dictionary = update_dist(merged_dictionary, _machine_counts[required_dictionary_index])
#     machine_counts.append(merged_dictionary)

# # Display results
# for idx, counts in enumerate(machine_counts):
#     print(f"Circuit {idx} counts: {counts}")


Running a total of 5 quantum objects in 1 batches


/var/folders/6h/gz0crzw532s0lr4pnj512s9c0000gn/T/ipykernel_8628/1240725466.py:4: DeprecationWarning: The module 'qiskit.test.mock' is deprecated since Qiskit Terra 0.21.0, and will be removed 3 months or more later. Instead, you should import the desired object directly 'qiskit.providers.fake_provider'.
  from qiskit.test.mock import FakeProvider


QiskitError: "Invalid input object QASM Qobj: e836f150-ef0b-4dc3-b01c-fd3f2cd0d6fc:\nConfig: {'init_qubits': True,\n 'meas_level': <MeasLevel.CLASSIFIED: 2>,\n 'memory': False,\n 'memory_slots': 3,\n 'n_qubits': 3,\n 'parameter_binds': [],\n 'parametric_pulses': ['gaussian', 'gaussian_square', 'drag', 'constant'],\n 'rep_delay': 250.0,\n 'shots': 8192}\nHeader: {'backend_name': 'fake_toronto', 'backend_version': '1.4.21'}\nExperiments:\n\nQASM Experiment:\nHeader:\n{'clbit_labels': [['meas', 0], ['meas', 1], ['meas', 2]],\n 'creg_sizes': [['meas', 3]],\n 'global_phase': 0.0,\n 'memory_slots': 3,\n 'metadata': {},\n 'n_qubits': 3,\n 'name': 'circuit-95',\n 'qreg_sizes': [['q', 3]],\n 'qubit_labels': [['q', 0], ['q', 1], ['q', 2]]}\nConfig:\n{'memory_slots': 3, 'n_qubits': 3}\n\n\tInstruction: h\n\t\tqubits: [0]\n\n\tInstruction: h\n\t\tqubits: [1]\n\n\tInstruction: h\n\t\tqubits: [2]\n\n\tInstruction: cx\n\t\tqubits: [0, 1]\n\n\tInstruction: cx\n\t\tqubits: [1, 2]\n\n\tInstruction: barr